In [53]:
import torch 
import torchvision
from torchvision import transforms
import torchvision.datasets as datasets
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
from glob import glob
from PIL import Image

In [54]:
df_data = pd.read_csv('HAM10000_metadata')
df_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [55]:
img_path = {os.path.splitext(os.path.basename(x))[0]: x for x in glob((os.path.join('*', '*.jpg')))}

df_data['img_path'] = df_data['image_id'].map(img_path.get)

#drop rows with no image path
df_data.dropna(inplace=True)
df_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,img_path
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,HAM10000_images_part_1\ISIC_0025030.jpg
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,HAM10000_images_part_1\ISIC_0026769.jpg
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,HAM10000_images_part_1\ISIC_0025661.jpg
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,HAM10000_images_part_2\ISIC_0031633.jpg
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back,vidir_modern,HAM10000_images_part_1\ISIC_0025837.jpg


In [56]:
def load_image(image_path):
    try:
        image = Image.open(image_path)
        return np.asarray(image.resize((32, 32)))
    except Exception as e:
        print(f"Error loading image '{image_path}': {e}")
        return None
    
def transpose(img):
    '''
    Apply transpose to an image such that the color channels are first
    '''
    return np.transpose(img, (2, 0 ,1))


In [57]:
# change image shape in entire dataset
df_data['img'] = df_data['img_path'].map(load_image)
df_data['img'] = df_data['img'].apply(transpose)

# drop na values 
df_data.dropna()

# take a look at dataset
df_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,img_path,img
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,HAM10000_images_part_1\ISIC_0025030.jpg,"[[[24, 56, 106, 143, 167, 173, 177, 178, 185, ..."
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,HAM10000_images_part_1\ISIC_0026769.jpg,"[[[190, 199, 200, 205, 207, 207, 209, 201, 199..."
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,HAM10000_images_part_1\ISIC_0025661.jpg,"[[[35, 83, 128, 161, 174, 180, 191, 192, 199, ..."
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,HAM10000_images_part_2\ISIC_0031633.jpg,"[[[155, 188, 210, 220, 228, 233, 235, 234, 238..."
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back,vidir_modern,HAM10000_images_part_1\ISIC_0025837.jpg,"[[[122, 158, 179, 184, 191, 188, 194, 195, 199..."


In [58]:
from sklearn.model_selection import train_test_split

# train test split
X_train, X_test, y_train, y_test = train_test_split(df_data[['lesion_id', 'img']], df_data['dx'], test_size=0.3, random_state=4567823)

# X_train has lesion_id and img
X_train.head()

,lesion_id,img
1762,HAM_0007102,"[[[194, 195, 193, 201, 198, 202, 210, 210, 206..."
1265,HAM_0002011,"[[[164, 178, 181, 185, 188, 192, 194, 195, 196..."
8726,HAM_0006367,"[[[173, 177, 181, 183, 187, 181, 185, 192, 196..."
9529,HAM_0004350,"[[[166, 176, 180, 184, 188, 190, 194, 193, 194..."
6795,HAM_0005237,"[[[196, 200, 204, 205, 209, 210, 211, 213, 210..."


In [59]:
# display the number of entries in each class
y_train.value_counts()


dx
nv       3722
mel       645
bkl       625
bcc       292
akiec     185
vasc       90
df         68
Name: count, dtype: int64

In [60]:
# lets consolidate the classes into mel and non-mel
# melanoma is the most dangerous type of skin cancer
# so we will consider it as mel

# lets create a dictionary to map the classes
lesion_type_dict = {
    'nv': 'Non-Mel',
    'mel': 'Mel',
    'bkl': 'Non-Mel',
    'bcc': 'Non-Mel',
    'akiec': 'Non-Mel',
    'vasc': 'Non-Mel',
    'df': 'Non-Mel'
}

# map the classes
y_train = y_train.map(lesion_type_dict)
y_test = y_test.map(lesion_type_dict)

# lets convert the classes to binary
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

# lets take a look at the classes
y_train.head()



,Mel,Non-Mel
1762,True,False
1265,True,False
8726,False,True
9529,False,True
6795,False,True


In [61]:
# lets look at the number of entries in each class
y_train.sum()

# print ratio of mel to non-mel
print(f"Mel to Non-Mel ratio: {y_train.sum()[0] / y_train.sum()[1]}")



Mel to Non-Mel ratio: 0.12946607788036932


C:\Users\julia\AppData\Local\Temp\ipykernel_26964\2898249329.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Mel to Non-Mel ratio: {y_train.sum()[0] / y_train.sum()[1]}")


In [74]:
# balance the training dataset by ignoring 88% of the non-mel samples
# this is done to balance the dataset

non_mel = y_train[y_train['Non-Mel'] == 1].sample(frac=0.12, random_state=1)
mel = y_train[y_train['Mel'] == 1]
y_train_even = pd.concat([non_mel, mel])
X_train_even = X_train.loc[y_train_even.index]

In [75]:
# display new ratio
print(f"Mel to Non-Mel ratio: {y_train.sum()[0] / y_train.sum()[1]}")
print(f"New Mel to Non-Mel ratio: {y_train_even.sum()[0] / y_train_even.sum()[1]}")

Mel to Non-Mel ratio: 0.12946607788036932
New Mel to Non-Mel ratio: 1.0785953177257526


C:\Users\julia\AppData\Local\Temp\ipykernel_26964\239130495.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Mel to Non-Mel ratio: {y_train.sum()[0] / y_train.sum()[1]}")
C:\Users\julia\AppData\Local\Temp\ipykernel_26964\239130495.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"New Mel to Non-Mel ratio: {y_train_even.sum()[0] / y_train_even.sum()[1]}")


Yay! now its even! back to the regular problem solving.

In [76]:
# lets convert the images to tensors
X_train_tensor = torch.tensor(X_train_even['img'].tolist())
X_test_tensor = torch.tensor(X_test['img'].tolist())

# lets take a look at the shape of the tensors
X_train_tensor.shape


torch.Size([1243, 3, 32, 32])

In [78]:
# lets create a dataloader
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(X_train_tensor, torch.tensor(y_train_even.values))
test_dataset = TensorDataset(X_test_tensor, torch.tensor(y_test.values))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# lets take a look at the train loader
for x, y in train_loader:
    print(x.shape, y.shape)
    break

# lets create a simple CNN model
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
# lets create an instance of the model
model = SimpleCNN()

# lets define the loss function and optimizer
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# lets train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000}')
            running_loss = 0.0

print('Finished Training')


torch.Size([32, 3, 32, 32]) torch.Size([32, 2])
Finished Training


In [79]:
# lets test the model
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.max(labels, 1)[1]).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct / total}%')


Accuracy of the network on the test images: 12.603648424543946%


In [80]:
# display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = []
y_true = []
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        y_pred.extend(predicted.numpy())
        y_true.extend(torch.max(labels, 1)[1].numpy())

confusion_matrix(y_true, y_pred)

array([[ 303,    0],
       [2108,    1]], dtype=int64)

This is wrong lol